In [8]:
#!/bin/env python3
import gdal, ogr, osr
import numpy as np
import os, sys
import os.path

# #the following three lines are for ZScore cumulative. Use these three lines, or the next three lines.
# infolder ='C:/Users/yapingxu.ASURITE/Documents/Allen Coral Atlas/Moorea_Weekly/Zscorefiles/'
# maskfile='C:/Users/yapingxu.ASURITE/Documents/Allen Coral Atlas/Moorea_Weekly/CoralNew/L15-0171E-0922N_coral2.tif'
# outfolder='C:/Users/yapingxu.ASURITE/Documents/Allen Coral Atlas/Moorea_Weekly/Zscorefiles_CoralMaskApplied/'


#the following three lines are for ZScore Weekly.
infolder ='C:/Users/yapingxu.ASURITE/Documents/Allen Coral Atlas/Moorea_Weekly/ZscoreWeekly/'
maskfile='C:/Users/yapingxu.ASURITE/Documents/Allen Coral Atlas/Moorea_Weekly/CoralNew/L15-0171E-0922N_coral2.tif'
outfolder='C:/Users/yapingxu.ASURITE/Documents/Allen Coral Atlas/Moorea_Weekly/ZscoreWeekly_CoralMaskApplied/'


#print(os.path.basename(maskfile))

for root,dirs,files in os.walk(infolder):
    for file in files:
        if file.endswith('.tif'):
            infile=os.path.join(root,file)
            outname="CoralMasked_"+os.path.basename(file)
            outfile=outfolder+outname
             
                
            def main(infile, maskfile, outfile):

                ## open inputfile
                if os.path.exists(infile):
                    inDS = gdal.Open(infile, gdal.GA_ReadOnly)
                else:
                    print("File %s does not exist" % (infile))
                    sys.exit(0)

                ## open maskfile
                if os.path.exists(maskfile):
                    maskDS = gdal.Open(maskfile, gdal.GA_ReadOnly)
                else:
                    inDS = None
                    print("File %s does not exist" % (maskfile))
                    sys.exit(0)


                ## get information from input file to match for output
                datatype = inDS.GetRasterBand(1).DataType
                proj = inDS.GetProjection()
                outXsize = inDS.RasterXSize
                outYsize = inDS.RasterYSize
                gt = inDS.GetGeoTransform()

                ## read mask image into memory
                mask = maskDS.GetRasterBand(1).ReadAsArray()
                maskDS = None



                ## read input data and create masks for setting non-coral areas to -9999
                indata = inDS.GetRasterBand(1).ReadAsArray()
                bad1 = np.logical_or(np.equal(indata, -9999), np.equal(indata, np.nan))
                bad2 = np.equal(mask, 0)
                bad = np.logical_or(bad1, bad2)
                indata[bad] = -9999

                ## create output file
                driver = gdal.GetDriverByName('GTiff')
                outDS = driver.Create(outfile, outXsize, outYsize, 1, datatype, options=['COMPRESS=LZW', 'TILED=YES']) 
                outDS.SetGeoTransform(gt)
                outDS.SetProjection(proj)
                outDS.GetRasterBand(1).SetNoDataValue(0)
                outDS.GetRasterBand(1).WriteArray(indata)


                ## write out data and close up the open files
                outDS.GetRasterBand(1).SetNoDataValue(-9999)
                #outDS.GetRasterBand(2).WriteArray(indata)
                outDS.FlushCache()

                inDS, outDS = None, None
            main(infile, maskfile, outfile)
                
                
        #获取文件路径
            

    
print('end')    
   # 
    #
   # 

    #print(os.path.abspath(outfile))       

end
